This code is based on the optimized structure of a homo-coupled 6-6 (akg) reaction in MenD (5EJ5) The idea is to keep the ThDP-bound product structure and use this code to only replace the R groups. 

In [1]:
import MDAnalysis as mda
from MDAnalysis.core.universe import Merge
import numpy as np
import os
from utils import *
import warnings 
import matplotlib.pyplot as plt
import subprocess

# Suppress warnings specific to MDAnalysis
warnings.filterwarnings("ignore", category=UserWarning, module="MDAnalysis")

In [2]:
# calculate vectors for the direction of each R group
# P denotes prime, the atom of the acceptor molecule 
C2_coords = np.array([54.162, 61.153, 67.179])
C2P_coords = np.array([53.460, 59.753, 66.866])
R_coords = np.array([53.151, 62.219, 67.658])
RP_coords = np.array([52.798, 58.997, 68.017])

C2_R_vec = R_coords - C2_coords
C2P_RP_vec = RP_coords - C2P_coords
C2R_unit = C2_R_vec / np.linalg.norm(C2_R_vec)
C2RP_unit = C2P_RP_vec / np.linalg.norm(C2P_RP_vec)

In [3]:
def calculate_average_plane(points):
    """
    Calculate the average plane through a group of points.
    Parameters:
        points (numpy.ndarray): A (N, 3) array where each row represents a 3D point.
    Returns:
        plane_normal (numpy.ndarray): The normal vector of the plane.
        plane_point (numpy.ndarray): A point on the plane (centroid of the points).
    """
    if points.shape[1] != 3:
        raise ValueError("Input points array must have shape (N, 3).")
    # Calculate the centroid of the points
    centroid = np.mean(points, axis=0)
    # Subtract the centroid to center the points
    centered_points = points - centroid
    # Perform SVD on the centered points
    _, _, vh = np.linalg.svd(centered_points)
    # The normal vector of the plane is the last column of vh
    plane_normal = vh[-1]

    return plane_normal, centroid

In [4]:
# define the coordinates of the heavy atom R groups for the donor and acceptor from the optimized geometry
# We will find the average plane through these points and try to match the normal of that plane when we replace 
# the new R groups for donor and acceptor

donor_heavy_atom_R_coords = np.array([[54.162, 61.153, 67.179],
                                      [53.151, 62.219, 67.658],
                                      [53.627, 63.697, 67.778],
                                      [52.602, 64.809, 67.849],
                                      [52.536, 65.447, 68.920],
                                      [51.655, 64.819, 67.061]])


acc_heavy_atom_R_coords =   np.array([[53.460, 59.753, 66.866],
                                      [52.798, 58.997, 68.017],
                                      [51.780, 57.864, 67.669],
                                      [50.325, 58.326, 67.625],
                                      [49.610, 57.990, 66.721],
                                      [49.879, 58.844, 68.664]])


# Calculate the average plane of our template R groups 
donor_normal, donor_centroid = calculate_average_plane(donor_heavy_atom_R_coords)
acc_normal, acc_centroid = calculate_average_plane(acc_heavy_atom_R_coords)

In [5]:
# Calculate the average plane of our template R groups 
donor_normal, donor_centroid = calculate_average_plane(donor_heavy_atom_R_coords)
acc_normal, acc_centroid = calculate_average_plane(acc_heavy_atom_R_coords)


In [6]:
# load receptor universe and extract the different parts of the protein int1 receptor 
head_dir = '5EJ5/int3_R/'
receptor = mda.Universe(head_dir+'template.pdb')

output_dir = '/projects/p30041/gbf4422/TEST_WORKFLOW/'

# output the ThDP cofactor
inp_base = receptor.select_atoms("resname INP")
#write_universe(output_dir +'receptor/','INP.pdb',ThDP)

# output just the protein and edit the file so it is Amber readable 
protein = receptor.select_atoms(f"protein or resname MG")
write_universe(output_dir  + 'receptor/','protein.pdb',protein)
edit_protein_files(output_dir + 'receptor/','protein.pdb')

# output the water
try:
    water = receptor.select_atoms(f"resname WAT")
    write_universe(output_dir + 'receptor/','water.pdb',water)
except:
    print('No water to write')

# output the receptor (everything besides ThDP)
protein_MG_water = receptor.select_atoms("not resname INP")
write_universe(output_dir  + 'receptor/','receptor.pdb',protein_MG_water)
edit_protein_files(output_dir  + 'receptor/','receptor.pdb')

Directory '/projects/p30041/gbf4422/TEST_WORKFLOW/receptor/' does not exist. Creating it...
File 'protein.pdb' has been written in '/projects/p30041/gbf4422/TEST_WORKFLOW/receptor/'.
Edited  /projects/p30041/gbf4422/TEST_WORKFLOW/receptor/protein.pdb  for Amber
File 'water.pdb' has been written in '/projects/p30041/gbf4422/TEST_WORKFLOW/receptor/'.
File 'receptor.pdb' has been written in '/projects/p30041/gbf4422/TEST_WORKFLOW/receptor/'.
Edited  /projects/p30041/gbf4422/TEST_WORKFLOW/receptor/receptor.pdb  for Amber


In [9]:
def diff_btwn_planes(normal_1,normal_2):
    # Normalize the normals
    n1 = normal_1 / np.linalg.norm(normal_1)
    n2 = normal_2 / np.linalg.norm(normal_2)
    
    # Calculate angle between normals
    dot_product = np.dot(n1, n2)
    angle = np.arccos(np.clip(dot_product, -1.0, 1.0))
    
    return angle

def rotation_objective(angle,md_universe,fixed_index_1,fixed_index_2,rotating_atom_indexes,reference_plane_normal,reference_plane_centroid):
    rotated_donor = rotate_atoms(md_universe, fixed_index_1,fixed_index_2,rotating_atom_indexes,angle)
    atom_positions = []
    for i in range(0,len(rotated_donor.atoms)):
        if rotated_donor.atoms[i].type != 'H':
            atom_positions.append(rotated_donor.atoms[i].position)
    plane_normal, plane_centroid = calculate_average_plane(np.array(atom_positions))
    angle_between_planes = diff_btwn_planes(plane_normal,reference_plane_normal)
    dist_between_centroid = get_dist(plane_centroid,reference_plane_centroid)
    return angle_between_planes+dist_between_centroid

def optimize_rotation(initial_angle,md_universe:mda.core.universe.Universe,fixed_index_1:int,fixed_index_2:int,rotating_atom_indexes:list,reference_plane_normal:np.ndarray,reference_plane_centroid:np.ndarray):
    # Set up optimization
    tolerance = 1e-12
    result = minimize(
        rotation_objective,
        initial_angle,
        args=(md_universe.copy(),fixed_index_1,fixed_index_2,rotating_atom_indexes,reference_plane_normal,reference_plane_centroid),
        tol=tolerance,
        method='Nelder-Mead'
    )
    # Check for successful optimization
    if result.success:
        print('ROTATION CONVERGED')
    else:
        print('ROTATION NOT CONVERGED')
    rotated_structure = rotate_atoms(md_universe.copy(), fixed_index_1,fixed_index_2,rotating_atom_indexes, result.x[0])
    #rotated_structure = rotate_atoms(md_universe.copy(), fixed_index_1,fixed_index_2,rotating_atom_indexes, 270)

    return rotated_structure

def find_atom_in_new_universe(md_universe,check_coords):
    min_dist = 10**6 
    for i in range(0,len(md_universe.atoms)):
        curr_pos = md_universe.atoms[i].position
        curr_dist = get_dist(curr_pos,check_coords)
        if curr_dist < min_dist:
            min_dist = curr_dist
            new_atom_index = i
    return new_atom_index

def double_rotation_objective(angles,md_universe,fixed_indexes_1,fixed_indexes_2,rotating_atom_indexes_1,rotating_atom_indexes_2):
    rotated_donor = rotate_atoms(md_universe.copy(), fixed_indexes_1[0],fixed_indexes_1[1],rotating_atom_indexes_1,angles[0])
    rotated_acc = rotate_atoms(rotated_donor.copy(), fixed_indexes_2[0],fixed_indexes_2[1],rotating_atom_indexes_2,angles[1])
    inp_rotated = rotated_acc.select_atoms("resname INP")
    
    check_atoms = [i.index for i in inp_rotated]
    num_clash = get_atom_clashes(rotated_acc,check_atoms,threshold=1.5)
    self_clash = get_atom_clashes(inp_rotated,check_atoms,threshold=1.5)

    print(num_clash)
    return num_clash

def align_and_update_positions(initial_coords,final_coords,md_universe):
    R, t = kabsch_algorithm(initial_coords,final_coords)
    # make a copy of the substrate object and update atom positions by aligning aka head atoms
    universe_aligned = md_universe.copy()
    for i in range(0,len(universe_aligned.atoms.positions)):
        atom_coords = universe_aligned.atoms[i].position
        new_coords = np.dot(R, atom_coords) + t
        universe_aligned.atoms[i].position = new_coords

    return universe_aligned

def optimize_double_rotation(initial_angles,md_universe,fixed_indexes_1,fixed_indexes_2,rotating_atom_indexes_1,rotating_atom_indexes_2):
    # Set up optimization
    tolerance = 1e-6
    result = minimize(
        double_rotation_objective,
        initial_angles,
        args=(md_universe,fixed_indexes_1,fixed_indexes_2,rotating_atom_indexes_1,rotating_atom_indexes_2),
        tol=tolerance,
        method='Nelder-Mead'
    )
    # Check for successful optimization
    if result.success:
        print('ROTATION CONVERGED')
    else:
        print('ROTATION NOT CONVERGED')
    #rotated_structure = rotate_atoms(md_universe.copy(), fixed_index_1,fixed_index_2,rotating_atom_indexes, result.x[0])
    #rotated_structure = rotate_atoms(md_universe.copy(), fixed_index_1,fixed_index_2,rotating_atom_indexes, 270)

    return result.x

For atom labeling scheme see the image int1_ThDP_atom_labeling_scheme.png found in this directory

In [10]:
# read in each of the substrate files
directory = 'substrates_initial/'
#file_names = [f for f in os.listdir(directory)]
donor_file_names = [f'{i}.pdb' for i in [0]]
acceptor_file_names = [f'{i}.pdb' for i in [3]]

for donor_file_name in donor_file_names:
    for acceptor_file_name in acceptor_file_names:
        print('Donor, Acceptor',donor_file_name,acceptor_file_name)
        donor_index = donor_file_name.split('.')[0]
        acc_index = acceptor_file_name.split('.')[0]
        output_dir = '/projects/p30041/gbf4422/TEST_WORKFLOW/' + donor_index + '_' + acc_index + '/MD/int3/'

        # load the donor substrate universe
        donor_substrate = mda.Universe(directory+donor_file_name)
        # identify the important atoms for alignment (alpha keto acid atoms) 
        donor_substrate_important_indexes = get_substrate_aka_indexes(donor_substrate.atoms)
        # keep track of the atoms to be deleted (alkoxide (already included in template) and carboxylic acid atoms (CO2 has left at this point in the reaction))
        donor_indices_to_remove = [donor_substrate_important_indexes['O1'],
                                    donor_substrate_important_indexes['C3'],
                                    donor_substrate_important_indexes['O2'],
                                    donor_substrate_important_indexes['O3']]
        
        # S denotes unbound susbtrate we are trying to align 
        C2S_coords = get_atom_position(donor_substrate,donor_substrate_important_indexes['C2'])
        RS_coords = get_atom_position(donor_substrate,donor_substrate_important_indexes['R'])
        # initial coordinates are just in space where the substrate was initially drawn (like in Avogadro) and need to be moved
        initial_S_positions = np.array([C2S_coords,RS_coords]) 
        # the first atom of the R groups I work with is always either a carbon or nitrogen, get the expected distance based on this
        if donor_index == '7':
            R_dist = bond_dists['C-N']
        else:
            R_dist = bond_dists['C-C']
        if acc_index == '7':
            RP_dist = bond_dists['C-N']
        else:
            RP_dist = bond_dists['C-C']

        guess_R_coords = C2_coords + C2R_unit * R_dist 
        final_donor_positions = np.array([C2_coords,guess_R_coords])
        
        # P denotes prime, an atom of the acceptor molecule 
        guess_RP_coords = C2P_coords + C2RP_unit * RP_dist 
        final_acc_positions = np.array([C2P_coords,guess_RP_coords])

        # align the donor substrate aka atoms to those of the template 
        donor_aligned = align_and_update_positions(initial_S_positions,final_donor_positions,donor_substrate)
        # Select atoms to keep (all atoms excluding the ones in indices_to_remove)
        mask = ~np.isin(donor_aligned.atoms.indices, donor_indices_to_remove)
        atoms_to_keep = donor_aligned.atoms[mask]
        modified_donor = Merge(atoms_to_keep)
        
        # we now need to rotate the R group of the substrate to fit into the active site 
        C2_new_index = find_atom_in_new_universe(modified_donor,C2_coords)
        R_new_index = find_atom_in_new_universe(modified_donor,R_coords)
        donor_atoms_to_rotate = [i for i in range(0,len(modified_donor.atoms)) if i not in [C2_new_index,R_new_index]]
        rotated_donor = optimize_rotation(np.array(0),modified_donor.copy(),C2_new_index, R_new_index, donor_atoms_to_rotate,donor_normal,donor_centroid)

        # Acceptor placement 
        acc_substrate = mda.Universe(directory+acceptor_file_name)
        
        # identify the important atoms for alignment (alpha keto acid atoms) 
        acc_substrate_important_indexes = get_substrate_aka_indexes(acc_substrate.atoms)
        # keep track of the atoms to be deleted (alkoxide and carboxylic acid atoms, all already included in template)
        acc_indices_to_remove = [acc_substrate_important_indexes['O1'],
                                    acc_substrate_important_indexes['C3'],
                                    acc_substrate_important_indexes['O2'],
                                    acc_substrate_important_indexes['O3']]
        
        # S denotes unbound susbtrate we are trying to align 
        C2S_coords = get_atom_position(acc_substrate,acc_substrate_important_indexes['C2'])
        RS_coords = get_atom_position(acc_substrate,acc_substrate_important_indexes['R'])
        # initial coordinates are just in space where the substrate was initially drawn (like in Avogadro) and need to be moved
        initial_S_positions = np.array([C2S_coords,RS_coords]) 
        
        
        acc_aligned = align_and_update_positions(initial_S_positions,final_acc_positions,acc_substrate)
        # Select atoms to keep (all atoms excluding the ones in indices_to_remove)
        mask = ~np.isin(acc_aligned.atoms.indices, acc_indices_to_remove)
        atoms_to_keep = acc_aligned.atoms[mask]
        modified_acc = Merge(atoms_to_keep)
        #write_universe(output_dir+file_start+'/', 'acc.pdb', modified_acc)
        C2P_new_index = find_atom_in_new_universe(modified_acc,C2P_coords)
        RP_new_index = find_atom_in_new_universe(modified_acc,RP_coords)
        acc_atoms_to_rotate = [i for i in range(0,len(modified_acc.atoms)) if i not in [C2P_new_index,RP_new_index]]
        rotated_acc = optimize_rotation(np.array(0),modified_acc.copy(),C2P_new_index, RP_new_index, acc_atoms_to_rotate,acc_normal,acc_centroid)
        #write_universe(output_dir+file_start+'/', 'rotated_acc.pdb', rotated_acc)

        inp = mda.Merge(inp_base.atoms,rotated_donor.atoms,rotated_acc.atoms)
        for atom in inp.atoms:
            atom.residue.resid = max(protein_MG_water.ids) + 1
            atom.residue.resname = "INP"
            atom.record_type = "HETATM"

        complex = mda.Merge(protein_MG_water.atoms,inp.atoms)

        inp_donor_atoms_to_rotate = []
        for i in donor_atoms_to_rotate:
            new_index = find_atom_in_new_universe(complex,rotated_donor.atoms[i].position)
            inp_donor_atoms_to_rotate.append(new_index)
        C2_complex_index = find_atom_in_new_universe(complex,rotated_donor.atoms[C2_new_index].position) 
        R_complex_index = find_atom_in_new_universe(complex,rotated_donor.atoms[R_new_index].position) 

        inp_acc_atoms_to_rotate = []
        for i in acc_atoms_to_rotate:
            new_index = find_atom_in_new_universe(complex,rotated_acc.atoms[i].position)
            inp_acc_atoms_to_rotate.append(new_index)
        C2P_complex_index = find_atom_in_new_universe(complex,rotated_acc.atoms[C2P_new_index].position) 
        RP_complex_index = find_atom_in_new_universe(complex,rotated_acc.atoms[RP_new_index].position) 

        #final_angles = optimize_double_rotation([90,0],complex,[C2_complex_index,R_complex_index],[C2P_complex_index,RP_complex_index],inp_donor_atoms_to_rotate,inp_acc_atoms_to_rotate)

        angles1 = np.linspace(0, 360, 30)
        angles2 = np.linspace(0, 360, 30)
        obj_vals = np.array([[double_rotation_objective([a1, a2],complex,[C2_complex_index,R_complex_index],[C2P_complex_index,RP_complex_index],inp_donor_atoms_to_rotate,inp_acc_atoms_to_rotate) for a2 in angles2] for a1 in angles1])
        min_indexes = np.unravel_index(np.argmin(obj_vals), obj_vals.shape)

        rotated_donor_complex = rotate_atoms(complex.copy(), C2_complex_index,R_complex_index,inp_donor_atoms_to_rotate,angles1[min_indexes[0]])
        rotated_complex = rotate_atoms(rotated_donor_complex.copy(), C2P_complex_index,RP_complex_index,inp_acc_atoms_to_rotate,angles2[min_indexes[1]])
        write_universe(output_dir+'prep/', 'initial_complex.pdb', rotated_complex)
        edit_protein_files(output_dir+'prep/','initial_complex.pdb')
        inp_final = rotated_complex.select_atoms("resname INP")
        write_universe(output_dir+'prep/', 'initial_inp.pdb', inp_final)
        
        inp_base_charge = -4 
        # add the charge of the intermediate complex
        if donor_index in ['4','6','11','16']:
            inp_base_charge += -1
        if acc_index in ['4','6','11','16']:
            inp_base_charge += -1
        
        
        # Run the bash script from the specified directory
        #result = subprocess.run(
        #    ["bash", "/projects/p30041/gbf4422/Generate_QMMM_Geometries/scripts/assemble_int3_for_MD.sh",str(inp_base_charge), output_dir+'prep/'],
        #    cwd=head_dir,
        #    capture_output=True,
        #    text=True
        #)
        #
        #if result.returncode != 0:
        #    print("Error: Script failed.")
        #    print(result.stderr)  # Print the error output
        #else:
        #    print('tleap was successful')
        #    #print(result.stdout)  # Print the normal output
        print('DONE')


Donor, Acceptor 0.pdb 3.pdb
ROTATION CONVERGED
ROTATION CONVERGED
File 'initial_complex.pdb' has been written in '/projects/p30041/gbf4422/TEST_WORKFLOW/0_3/MD/int3/prep/'.
Edited  /projects/p30041/gbf4422/TEST_WORKFLOW/0_3/MD/int3/prep/initial_complex.pdb  for Amber
File 'initial_inp.pdb' has been written in '/projects/p30041/gbf4422/TEST_WORKFLOW/0_3/MD/int3/prep/'.


In [ ]:
# Run the bash script from the specified directory
result = subprocess.run(
    ["bash", "/projects/p30041/gbf4422/Generate_QMMM_Geometries/scripts/assemble_int3_for_MD.sh",str(inp_base_charge), output_dir+'prep/'],
    cwd=head_dir,
    capture_output=True,
    text=True
)

if result.returncode != 0:
    print("Error: Script failed.")
    print(result.stderr)  # Print the error output
else:
    print('tleap was successful')
    #print(result.stdout)  # Print the normal output

In [ ]:
type(tmp)


In [ ]:
obj_vals

In [ ]:
plt.contourf(angles1, angles2, obj_vals, levels=50)
plt.colorbar()
plt.xlabel("Angle 1")
plt.ylabel("Angle 2")
plt.title("Objective Function Landscape")
plt.show()

    

In [ ]:
min_index = np.unravel_index(np.argmin(obj_vals), obj_vals.shape)
min_index

In [ ]:
obj_vals[2,6]